In [0]:
#GPUチェック
!nvidia-smi

In [0]:
#ABEJAコマンドのパッケージダウンロード
! curl -s https://packagecloud.io/install/repositories/abeja/platform-public/script.python.sh | bash

In [0]:
pip install abejacli

In [0]:
# コマンドの設定
! abeja config init

In [0]:
!pip install --upgrade mxnet-cu100 tqdm

In [0]:
!pip install autogluon==0.0.8

※　このタイミングでランタイムの再起動を実行する必要がある

In [0]:
import autogluon as ag
from autogluon import ObjectDetection as task

In [0]:
root = './'
filename_zip = ag.download('https://autogluon.s3.amazonaws.com/datasets/tiny_motorbike.zip',path=root)
filename = ag.unzip(filename_zip, root=root)

In [0]:
import os
data_root = os.path.join(root, filename)
dataset_train = task.Dataset(data_root, classes=('motorbike',))

In [0]:
time_limits = 5*60*60  # 5 hours
epochs = 10
detector = task.fit(dataset_train,
                    num_trials=1,
                    batch_size=8,
                    epochs=epochs,
                    lr=ag.Categorical(5e-4, 1e-4),
                    ngpus_per_trial=1,
                    time_limits=time_limits)

In [0]:
dataset_test = task.Dataset(data_root, index_file_name='test', classes=('motorbike',))
test_map = detector.evaluate(dataset_test)
print("mAP on test dataset: {}".format(test_map[1][1]))

In [0]:
image = '000873.jpg'
image_path = os.path.join(data_root, 'JPEGImages', image)
ind, prob, loc = detector.predict(image_path)

In [0]:
from google.colab import files
uploaded = files.upload()

モデルのsaveとload

In [0]:
from autogluon import Detector
savefile = 'model.pkl'
detector.save(savefile)

In [0]:
load_detector = Detector.load(savefile)